# Create the model

In [128]:
import os
from cntk import default_options, input_variable, ModelFormat
from cntk.layers import Dense, Sequential
from cntk.ops import log_softmax, relu, sigmoid

In [129]:
model = Sequential([
    Dense(4, activation=sigmoid),
    Dense(3, activation=log_softmax)
])

## Define the input for the neural network
The input for the model is a vector with four features:
 
 - Sepal length
 - Sepal width
 - Petal length
 - Petal width
 
In order for the model to work we need to define its input as an `input_variable`. This variable should have the same size as the number of features that we want to use for making a prediction. In this case it should be 4, because we have 4 different features in our dataset.

In [130]:
features = input_variable(4)

## Finalize the neural network structure
The last step is to finalize the neural network structure. We define a new variable `z` and invoke the model function with the input variable to bind it as the input for our model. 

In [131]:
z = model(features)

# Train the model and record it in the workspace
After we've created the model we can train it. We'll train the model and track it using the tracking logic provided by the Azure Machine Learning Environment.

## Loading the data
Before we can actually train the model, we need to load the data from disk. We will use pandas for this.
Pandas is widely used python library for working with data. It contains functions to load and process data 
as well as a large amount functions to perform statistical operations.

In [132]:
import pandas as pd

In [133]:
df_source = pd.read_csv('iris.csv', 
    names=['sepal_length', 'sepal_width','petal_length','petal_width', 'species'], 
    index_col=False)

In [134]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [135]:
df_source.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


We split the dataset into features `X` and labels `y`. We need to feed these separately to the trainer later on to train the model. We convert the features and labels to numpy arrays as this is what CNTK expects as input.

In [136]:
import numpy as np

In [137]:
X = df_source.iloc[:, :4].values
y = df_source.iloc[:, -1:].values

Our model doesn't take strings as values. It needs floating point values to do its job. So we need to encode the strings into a floating point representation. We can do this using a standard label encoder which is available in the `scikit-learn` python package.

In [138]:
from sklearn.preprocessing import LabelBinarizer

In [139]:
label_encoder = LabelBinarizer()

In [140]:
y = label_encoder.fit_transform(y)

CNTK is configured to use 32-bit floats by default. Right the features are stored as 64-bit floats and the labels are stored as integers. In order to help CNTK make sense of this, we will have to convert our data to 32-bit floats.

In [141]:
X = X.astype(np.float32)
y = y.astype(np.float32)

One of the challenges with machine learning is the fact that your model will try to memorize every bit of data it saw. This is called overfitting and bad for your model as it is no longer able to correctly predict outcome correctly for samples it didn't see before. We want our model to learn a set of rules that predict the correct class of flower. 

In order for us to detect overfitting we need to split the dataset into a training and test set. This is done using a utility function found in the scikit-learn python package which is included with your standard anaconda installation.

In [142]:
from sklearn.model_selection import train_test_split

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

## Defining the target and loss
Let's define a target for our model and a loss function. The loss function measures the distance between the actual and predicted value. The loss is later used by the learner to optimize the parameters in the model.

In [144]:
from cntk.losses import cross_entropy_with_softmax
from cntk.metrics import classification_error

In [145]:
label = input_variable(3)

In [146]:
loss = cross_entropy_with_softmax(z, label)

In [147]:
error_rate = classification_error(z, label)

## Setting up the learner and trainer
When we have a model and loss we can setup the learner and trainer to train the model.
We first define the learner, which is going to use the loss function and target to optimize the model.

In [148]:
from cntk.learners import sgd
from cntk.train.trainer import Trainer

In [149]:
learner = sgd(z.parameters, 0.001)

In [150]:
trainer = Trainer(z, (loss, error_rate), [learner])

# Train the model
We can train the model as normal. In order to track information about the model we need to setup a workspace and experiment in the Azure Machine Learning workspace that we've configured in the `config.json` in the same folder as this notebook. Please refer to chapter 7, Deploying models to production, to learn more on how to create this file.

In [151]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment = Experiment(name='classify-flowers', workspace=ws)

Found the config file in: D:\projects\cntk-book\ch7\azure-ml-service\config.json


We can start tracking methods by calling the `start_logging` method on the experiment. This starts a new run instance that has all the tracking logic that we need for our experiment. We can use `log` to track metrics. We can also use `upload_file` to store outputs generated by our run. And finally we can register uploaded files as models in the model registry so we can deploy them to production.

In [152]:
os.makedirs('outputs', exist_ok=True)

with experiment.start_logging() as run:
    for _ in range(10):
        trainer.train_minibatch({ features: X_train, label: y_train })

        run.log('average_loss', trainer.previous_minibatch_loss_average)
        run.log('average_metric', trainer.previous_minibatch_evaluation_average)
        
    test_metric = trainer.test_minibatch( {features: X_test, label: y_test })
    
    run.log('test_metric', test_metric)
    
    z.save('outputs/model.onnx', ModelFormat.ONNX)
    run.upload_file('model.onnx', 'outputs/model.onnx')
    
    stored_model = run.register_model(model_name='classify_flowers', model_path='model.onnx')

## Deploy the model to production
Now that we have a trained model we can deploy it to production. We need to setup an image for this and a deploy the image as a webservice to the cloud. Let's start with the image first.

In [153]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script="score.py", 
    runtime="python", 
    conda_file="conda_env.yml")

Once we have the configuration for the image we can invoke deploy_from_model with a deployment configuration to deploy the model as a Azure container instance to the cloud.

In [155]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Webservice.deploy_from_model(workspace=ws,
                                       name='classify-flowers-svc-2',
                                       deployment_config=aciconfig,
                                       models=[stored_model],
                                       image_config=image_config)

Creating image
Image creation operation finished for image classify-flowers-svc-2:1, operation "Succeeded"
Creating service
